In [299]:
import cv2
import numpy as np
import os
import glob
from numpy import linalg as la
import math

Parameters

In [300]:
# focal length
FOCAL_LENGTH = 668
# cylinder radius, it's much better to use focal length
CYLINDER_RADIUS = FOCAL_LENGTH
RANSAC_THRESHHOLD = 20


In [301]:
# show img for testing
def show_image_by_OpenCV(img):
    cv2.imshow('My Image', img )
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [302]:
def load_images_from_folder(folder):
    paths = glob.glob(os.path.join(folder,'*'))
    images = [cv2.imread(i) for i in paths]
    if(images[0].shape[0] > 2000):
        images = [cv2.resize(i, (i.shape[0]//10, i.shape[1]//10), interpolation=cv2.INTER_AREA) for i in images]
    return images

In [303]:
def nonmax_suppression(image):

    NONMAX_SUPPRESSION_KERNEL = 15
    STRIDE = 15
    
    result = np.zeros((image.shape))
    
    _image = np.lib.stride_tricks.sliding_window_view(image, NONMAX_SUPPRESSION_KERNEL, 1)[:,::STRIDE]
    _image = np.lib.stride_tricks.sliding_window_view(_image, NONMAX_SUPPRESSION_KERNEL, 0)[::STRIDE,:]
    # print(_image.shape)
    _image = np.reshape(_image,(*_image.shape[:2],-1))
    _image = np.argmax(_image, axis=2)

    mg = np.mgrid[0:_image.shape[0],0:_image.shape[1]]

    # y offset: mg[0], x offset: mg[1]
    index_y = _image // NONMAX_SUPPRESSION_KERNEL + mg[0] * STRIDE 
    index_x = _image % NONMAX_SUPPRESSION_KERNEL + mg[1] * STRIDE 
    index_x, index_y = index_x.flatten(), index_y.flatten()


    for i, j in zip(index_y, index_x):
        result[i][j]=image[i][j]  
        
    return  result

In [304]:
def detect_Harris_corner(image):
    # parameters
    KERNEL_SIZE = 3
    THRESHOLD = 100
    K = 0.04

    #  Compute x and y derivatives of image.
    # rgb to grayscale
    Gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    # gaussion : to do small smooth
    Gau_img = cv2.GaussianBlur(Gray_img, (5,5),1)
    # gradient
    I_y = np.gradient(Gau_img, axis=0) 
    I_x = np.gradient(Gau_img, axis=1) 


    # Compute products of derivates at each pixel.
    I_xx = I_x * I_x
    I_yy = I_y * I_y
    I_xy = I_x * I_y

    # Compute the sums of the products of derivates at each pixel.
    # weight : use gaussian
    S_xx = cv2.GaussianBlur(I_xx, (KERNEL_SIZE,KERNEL_SIZE),1.5)
    S_yy = cv2.GaussianBlur(I_yy, (KERNEL_SIZE,KERNEL_SIZE),1.5)
    S_xy = cv2.GaussianBlur(I_xy, (KERNEL_SIZE,KERNEL_SIZE),1.5)
    
    # Define the matrix M.
    # | S_xx  S_xy |
    # | S_xy  S_yy |

    # Compute the response of the detector at each pixel
    detM = S_xx * S_yy - S_xy * S_xy
    traceM = S_xx + S_yy
    R = detM - K * (traceM * traceM)

    # Nonmax Suppression
    R = nonmax_suppression(R)
    
    # Threshold on value of R
    R[R < THRESHOLD] = 0
    R[R > 0] = 255

    
    # 濾邊界
    R[:20 , :] = 0
    R[: ,:20 ] = 0
    R[-20:, :] = 0
    R[:,-20: ] = 0
    # return mask
    # print(R.shape)
    return R
        

In [305]:
def feature_descriptor(feature_mask, image):

    Gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    gau_img = cv2.GaussianBlur(Gray_img, (3,3), 4.5)
    sobel_y = cv2.Sobel(gau_img, cv2.CV_64F, 0, 1)
    sobel_x = cv2.Sobel(gau_img, cv2.CV_64F, 1, 0)
    _, angle = cv2.cartToPolar(sobel_x, sobel_y, angleInDegrees=True)
    # print(angle.dtype)
    DESCRIPTOR_SIZE = 40
    KERNEL = 5

    # descriptor size: 40*40
    feature_point_y, feature_point_x = np.nonzero(feature_mask)
    index_to_get = [KERNEL//2 + i * KERNEL for i in range(DESCRIPTOR_SIZE//KERNEL)]
    # 5*5 sum/mean -> 8*8(descriptor matrix)

    descriptors = []
    positions = []
    for y, x in zip(feature_point_y, feature_point_x):
        y, x = int(y), int(x)
        rot_M = cv2.getRotationMatrix2D((x,y),angle[y,x],1) # (旋轉中心),旋轉角度,縮放比例
        img_rotate = cv2.warpAffine(gau_img,rot_M,(gau_img.shape[1],gau_img.shape[0]), flags = cv2.INTER_NEAREST)

        # 取feature_point-20~+19之間的值，做boxfilter
        big_matrix = img_rotate[y-DESCRIPTOR_SIZE//2:y+DESCRIPTOR_SIZE//2, x-DESCRIPTOR_SIZE//2:x+DESCRIPTOR_SIZE//2]
        big_matrix = cv2.boxFilter(big_matrix, -1, (KERNEL, KERNEL))
        # print(big_matrix.dtype)
        # size = (8, 8)
        descriptor = np.zeros((DESCRIPTOR_SIZE//KERNEL, DESCRIPTOR_SIZE//KERNEL))
        for i in range(DESCRIPTOR_SIZE//KERNEL):
            for j in range(DESCRIPTOR_SIZE//KERNEL):
                descriptor[i][j] = big_matrix[index_to_get[i]][index_to_get[j]]
        descriptor = (descriptor - np.mean(descriptor)) / np.std(descriptor)
        descriptors.append(descriptor)
        positions.append([y, x])

    return np.array(descriptors), np.array(positions)

In [306]:
def find_match_poing(descriptors_1, positions_1, descriptors_2, positions_2, LOW_THERESHOLD=5.0):
    match_point = []
    # match_dis = []
    
    for ds1, ps1 in zip(descriptors_1, positions_1):
        min_dis = 99999
        min_pos = [-1, -1]

        for ds2, ps2 in zip(descriptors_2, positions_2):
            dis = np.linalg.norm(ds1 - ds2)

            if(dis<min_dis):
                min_dis = dis
                min_pos = np.array([*ps1, *ps2])
                
        if(min_dis<LOW_THERESHOLD):
            match_point.append(min_pos)
            # match_dis.append(min_dis)
    return match_point

In [307]:
def matching(descriptors_1, positions_1, descriptors_2, positions_2):
    # print('find_match_poing1')
    match_point_to_1 = find_match_poing(descriptors_1, positions_1, descriptors_2, positions_2)
    
    # print('find_match_poing2')
    match_point_to_2 = find_match_poing(descriptors_2, positions_2, descriptors_1, positions_1)

    # print('merge')
    match_point = []
    for match_1 in match_point_to_1:
        for match_2 in match_point_to_2:
            dis_1 = np.linalg.norm(match_1[:2]-match_2[-2:])
            dis_2 = np.linalg.norm(match_1[-2:]-match_2[:2])
            
            if(dis_1+dis_2==0):
                match_point.append(match_1)
                
    return np.array(match_point)
    
    # [pos1_y, pos1_x, pos2_y, pos2_x]
    

In [308]:
def draw_point(img, R, output, outputName, r, g, b):
    # img = np.copy(img)
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            if (R[i][j]>1):
                img = cv2.circle(img, (j, i), 2, (b, g, r), 2)
    if(output):
        cv2.imwrite(f"{outputName}.jpg", img)

In [309]:
def output_matching_image(images):
    print('Detecting Harris corner.')
    imgs_detect_Harris_corner = [detect_Harris_corner(i) for i in images]
    
    print('Genetating images feature descriptor.')
    
    for i, j in zip(images, imgs_detect_Harris_corner):
        # if drawpoint:
        draw_point(i, j, 0, '', 255, 0, 0)
    
    dsps = [feature_descriptor(i, j)for i,j in zip(imgs_detect_Harris_corner, images)]
    match_points = []

    print('Matching descriptor.')
    for i in range(1, len(images)):
        match_points.append(matching(dsps[i-1][0], dsps[i-1][1], dsps[i][0], dsps[i][1]))
    return match_points

In [310]:
def draw_match_line(imgs, match_points):
    for i in range(len(match_points)): 
        img_concat = np.concatenate((imgs[i], imgs[i+1]), axis=1)
        img_w = imgs[i].shape[1]
        for j in match_points[i]:
            img_concat = cv2.line(img_concat, (j[1], j[0]), (j[3]+img_w, j[2]), (255,0,0), 1)

        cv2.imwrite(f'result{i}_{i+1}.jpg', img_concat)

In [311]:
# it's better to use random sample
def nonRANSAC(mp, th = 2): # mp = match pairs set of all images
    best_mp = []
    
    for i in range(len(mp)): # every pairs of different img img_x & img_x+1
        best_vote = 0
        best_mp.append(mp[i][0])
        for j in range(len(mp[i])): # find the best pair of the two images
            vote = 0
            temp_pair = np.array(mp[i][:, 0:2])
            trans_y = mp[i][j][2] - mp[i][j][0]
            trans_x = mp[i][j][3] - mp[i][j][1]
            temp_pair[0] += trans_y 
            temp_pair[1] += trans_x
            for k in range(len(temp_pair)):
                if(k == j):
                    continue
                if(la.norm(temp_pair[k]) < th):
                    vote += 1

            if(vote > best_vote):
                best_vote = vote
                best_mp[i] = mp[i][j]

    return best_mp

Get Data from folder

In [312]:
# imgspath = glob.glob(os.path.join('img/gym/2-2','*.jpg'))
# imgs = [cv2.imread(i) for i in imgspath]

In [313]:
# img = imgs[0]
# new_img = np.zeros(img.shape)
# project_index = np.zeros(img.shape[0:2])
# project_index.shape

In [314]:
# img = imgs[0]
# img = cv2.resize(img, (img.shape[1]//10, img.shape[0]//10))
# new_img = np.zeros(img.shape)


In [315]:
def project_to_cylinder(img, f = FOCAL_LENGTH, s = CYLINDER_RADIUS):
    height, width = img.shape[:2]
    cylinder_proj = np.zeros(shape=img.shape, dtype=np.uint8)
    
    for y in range(height):
        py = -y + height//2
        for x in range(width):
            px = x - width//2
            cylinder_proj[-int(s*py/math.sqrt(px**2+f**2) - height//2)][int(s*math.atan(px/f) + width//2)] = img[y][x]
    
    return cylinder_proj

In [316]:
def crop_image_black_edge(img, f = FOCAL_LENGTH, s = CYLINDER_RADIUS):
    h, w = img.shape[:2]
    top = -int(s*(h//2)/math.sqrt((-w//2)**2+f**2) - h//2)
    down = -int(s*(-h//2)/math.sqrt((-w//2)**2+f**2) - h//2)
    left = int(s*math.atan((-w//2)/f) + w//2)
    right = int(s*math.atan((w//2)/f) + w//2)
    crop_img = img[top:down, left:right] # crop 4 sides
    # crop_img = img[:, left:right] # only crop 2 sides, left to right

    return crop_img

In [317]:
# it's better to use random sample
def nonRANSAC(mp, th = RANSAC_THRESHHOLD): # mp = match pairs set of all images
    best_mp = []

    for i in range(len(mp)): # every pairs of different img img_x & img_x+1
        best_vote = 0
        best_dis = 99999
        
        best_mp.append(mp[i][0])
        for j in range(len(mp[i])): # find the best pair of the two images
            vote = 0
            total_dis = 0
            temp_pair = np.array(mp[i][:, :2])
            trans_y = mp[i][j][2] - mp[i][j][0]
            trans_x = mp[i][j][3] - mp[i][j][1]
            # print("pair = ({}, {}, {}, {})".format(mp[i][j][0], mp[i][j][1], mp[i][j][2], mp[i][j][3]))
            # print("trans = {}, {}".format(trans_y, trans_x))
            temp_pair[:, 0] = temp_pair[:, 0] + trans_y - mp[i][:, 2]
            temp_pair[:, 1] = temp_pair[:, 1] + trans_x - mp[i][:, 3]
            # print("temp_pair.len = {}".format(len(temp_pair)))
            for k in range(len(temp_pair)):
                # print("k_pair = {}, norm = {}".format(temp_pair[k], la.norm(temp_pair[k])))
                if(la.norm(temp_pair[k]) < th):
                    vote += 1
                    total_dis += la.norm(temp_pair[k])**2



            if(vote >= best_vote):
                if(total_dis < best_dis and total_dis > 0):
                    best_dis = total_dis
                    best_vote = vote
                    best_mp[i] = mp[i][j]


    # for z in range(len(best_mp)):
    #     print("mp[0]: {}, best[0]: {}".format(mp[z][0], best_mp[z]))

    return best_mp
        

In [318]:
#  img2       img1            img2+img1
#  _____      _____           _____
# |     |    |.    |  __\    |    _|___   shift = (c,d) - (a,b)
# |    .|    |     |     \   |   |.|   |
# |_____|    |_____|  __ /   |___|_|   |
#  (c,d)      (a,b)     /        |_____|
# 
# y shift > 0, means img1 move down
# x shift > 0, means img1 move right
# in our case, our image are counterclockwise and equal size, so x shift must > 0

def stitch_image(img1, img2, shift): # img1 may be bigger, img2 is always the same size
    img1_padding = [
        (shift[0], 0) if shift[0] > 0 else (0, -shift[0]), 
        (shift[1], 0) if shift[1] > 0 else (0, -shift[1]), 
        (0, 0)
    ]
    padded_img1 = np.lib.pad(img1, img1_padding, 'constant', constant_values=0)

    # h2p = padded_img1.shape[0] - img2.shape[0]
    # w2p = padded_img1.shape[1] - img2.shape[1]
    # img2_padding = [
    #     (h2p, 0) if shift[0] < 0 else (0, -h2p), 
    #     (w2p, 0) if shift[1] < 0 else (0, -w2p), 
    #     (0, 0)
    # ]
    # padded_img2 = np.lib.pad(img2, img2_padding, 'constant', constant_values=0)


    padded_img1[:img2.shape[0], :img2.shape[1]] = img2

    return padded_img1
    # for i in range(len(match_points)): 
    #     img_concat = imgs[i+1]

    #     img_w = imgs[i].shape[1]
    #     for j in match_points[i]:
    #         img_concat = cv2.line(img_concat, (j[1], j[0]), (j[3]+img_w, j[2]), (255,0,0), 1)

    #     cv2.imwrite(f'result{i}_{i+1}.jpg', img_concat)

Main

In [319]:
# load images
images = load_images_from_folder('img/csie')
# images = load_images_from_folder('img/gym/2-2')

cylinder_images = [crop_image_black_edge(project_to_cylinder(i)) for i in images]

# return match point 
match_points = output_matching_image(cylinder_images)

ransac_mp = nonRANSAC(match_points)
shifts = [ransac_mp[i][2:] - ransac_mp[i][:2] for i in range(len(ransac_mp))]
for t in range(1, len(ransac_mp)):
    temp_img = stitch_image(cylinder_images[t-1], cylinder_images[t], shifts[t])
    cv2.imwrite(f"shift_result{t-1}_{t}.jpg", temp_img)
# draw_match_line(images, match_points)

Detecting Harris corner.
Genetating images feature descriptor.
Matching descriptor.


In [320]:
match_points[12][:]

array([[ 26,  80,  29, 316],
       [ 39, 344, 338,  23],
       [ 82,  31,  88, 268],
       [ 87, 102,  87, 343],
       [104,  83, 209, 323],
       [105, 105, 111, 345],
       [108,  89, 101, 329],
       [134,  25, 134, 268],
       [150,  81, 270, 320],
       [177,  73, 180, 316],
       [179, 113, 131, 352],
       [188,  79, 250, 320],
       [191, 102, 165, 343],
       [206, 103, 203, 345],
       [315,  90, 318, 331],
       [337, 306, 396, 186],
       [345, 105, 345, 348],
       [388, 145, 361, 291]])

In [321]:
temp_pair = np.array(match_points[0][:, :2])
print(temp_pair)
print(match_points[0][:,1])

temp_pair[:,0] += match_points[0][:,1]
temp_pair

[[ 40  28]
 [ 74 195]
 [126  97]
 [161 169]
 [176 132]
 [205  45]
 [238  23]
 [270 210]
 [315  30]
 [315  75]
 [363  63]
 [377  38]
 [407  46]]
[ 28 195  97 169 132  45  23 210  30  75  63  38  46]


array([[ 68,  28],
       [269, 195],
       [223,  97],
       [330, 169],
       [308, 132],
       [250,  45],
       [261,  23],
       [480, 210],
       [345,  30],
       [390,  75],
       [426,  63],
       [415,  38],
       [453,  46]])

In [322]:
mp = match_points
best_mp = []

for i in range(len(mp)): # every pairs of different img img_x & img_x+1
    best_vote = 0
    best_mp.append(mp[i][0])
    for j in range(len(mp[i])): # find the best pair of the two images
        vote = 0
        temp_pair = np.array(mp[i][:, :2])
        trans_y = mp[i][j][2] - mp[i][j][0]
        trans_x = mp[i][j][3] - mp[i][j][1]
        print("pair = ({}, {}, {}, {})".format(mp[i][j][0], mp[i][j][1], mp[i][j][2], mp[i][j][3]))
        print("trans = {}, {}".format(trans_y, trans_x))
        temp_pair[:, 0] = temp_pair[:, 0] + trans_y - mp[i][:, 2]
        temp_pair[:, 1] = temp_pair[:, 1] + trans_x - mp[i][:, 3]
        print("temp_pair.len = {}".format(len(temp_pair)))
        for k in range(len(temp_pair)):
            print("k_pair = {}, norm = {}".format(temp_pair[k], la.norm(temp_pair[k])))
            if(la.norm(temp_pair[k]) < 1):
                vote += 1

        if(vote > best_vote):
            best_vote = vote
            best_mp[i] = mp[i][j]

for z in range(len(best_mp)):
    print("mp[0]: {}, best[0]: {}".format(mp[z][0], best_mp[z]))

pair = (40, 28, 106, 319)
trans = 66, 291
temp_pair.len = 13
k_pair = [0 0], norm = 0.0
k_pair = [-332  186], norm = 380.5522303179946
k_pair = [60 47], norm = 76.21679604916491
k_pair = [ 69 116], norm = 134.97036711811967
k_pair = [-157   83], norm = 177.5894140989265
k_pair = [61 49], norm = 78.24321056807422
k_pair = [60 44], norm = 74.4043009509531
k_pair = [193 175], norm = 260.5263902179585
k_pair = [62 47], norm = 77.80102827083971
k_pair = [ 67 160], norm = 173.46181135915768
k_pair = [61 46], norm = 76.40026177965623
k_pair = [60 46], norm = 75.6042326857432
k_pair = [61 46], norm = 76.40026177965623
pair = (74, 195, 472, 300)
trans = 398, 105
temp_pair.len = 13
k_pair = [ 332 -186], norm = 380.5522303179946
k_pair = [0 0], norm = 0.0
k_pair = [ 392 -139], norm = 415.9146547069483
k_pair = [401 -70], norm = 407.06387705125593
k_pair = [ 175 -103], norm = 203.0615670184784
k_pair = [ 393 -137], norm = 416.19466599176883
k_pair = [ 392 -142], norm = 416.9268520975832
k_pair = [

In [323]:
ransac_mp = nonRANSAC(match_points)
ransac_mp

[array([363,  63, 368, 308]),
 array([220,  21, 224, 265]),
 array([201,  80, 205, 323]),
 array([161,  60, 164, 309]),
 array([206, 108, 208, 352]),
 array([239, 149, 234, 206]),
 array([172,  41, 175, 284]),
 array([164,  34, 165, 285]),
 array([ 25,  33,  29, 276]),
 array([135,  60, 137, 302]),
 array([ 29,  62,  91, 318]),
 array([ 65,  46,  71, 290]),
 array([ 87, 102,  87, 343]),
 array([172, 260, 289,  34]),
 array([171,  91, 177, 336]),
 array([234,  45, 238, 288]),
 array([268,  27, 267, 281])]

In [324]:
ransac_mp = nonRANSAC(match_points)
shifts = [ransac_mp[i][2:] - ransac_mp[i][:2] for i in range(len(ransac_mp))]
print(shifts)

[array([  5, 245]), array([  4, 244]), array([  4, 243]), array([  3, 249]), array([  2, 244]), array([-5, 57]), array([  3, 243]), array([  1, 251]), array([  4, 243]), array([  2, 242]), array([ 62, 256]), array([  6, 244]), array([  0, 241]), array([ 117, -226]), array([  6, 245]), array([  4, 243]), array([ -1, 254])]


In [325]:


cv2.imwrite("my_projection1.jpg", cylinder_images[0])
cv2.imwrite("my_projection2.jpg", cylinder_images[1])


True

In [326]:
nonRANSAC(match_points)

[array([363,  63, 368, 308]),
 array([220,  21, 224, 265]),
 array([201,  80, 205, 323]),
 array([161,  60, 164, 309]),
 array([206, 108, 208, 352]),
 array([239, 149, 234, 206]),
 array([172,  41, 175, 284]),
 array([164,  34, 165, 285]),
 array([ 25,  33,  29, 276]),
 array([135,  60, 137, 302]),
 array([ 29,  62,  91, 318]),
 array([ 65,  46,  71, 290]),
 array([ 87, 102,  87, 343]),
 array([172, 260, 289,  34]),
 array([171,  91, 177, 336]),
 array([234,  45, 238, 288]),
 array([268,  27, 267, 281])]